In [399]:
import numpy as np
from numpy import random as rand

In [400]:
def create_matrix(n_rows, n_columns):
    scores_array = rand.uniform(1,10,size=(n_rows, n_columns))
    return scores_array
 

In [401]:
# Test function
scores = create_matrix(4,4)

print(scores)

[[2.88144295 7.75572267 8.2596561  9.95275555]
 [2.65102091 9.45806778 9.32238493 3.426355  ]
 [6.36835243 8.71455767 1.50715765 3.61784713]
 [7.96453529 2.79092004 3.40903609 3.86891039]]


In [402]:
def calculate_scores_for_prediction(scores_array, percent):
    scores_for_prediction = scores_array.copy()
    number_of_zeros = int(np.round((1-percent)*scores_array.shape[0]*scores_array.shape[1]))
    choices = np.random.choice(scores_array.size, number_of_zeros, replace=False)
    scores_for_prediction.ravel()[choices] = 0 
    
    return scores_for_prediction

In [440]:
# Test function
scores_predict = calculate_scores_for_prediction(scores, 0.8)

print(scores_predict)

[[2.88144295 7.75572267 8.2596561  9.95275555]
 [2.65102091 9.45806778 9.32238493 3.426355  ]
 [6.36835243 8.71455767 0.         0.        ]
 [7.96453529 2.79092004 3.40903609 0.        ]]


In [441]:
def Jaccard_Similarity(x, y):
    intersection = len(set(x) & set(y))
    union = len(set(x) | set(y))
    return intersection/float(union)

In [442]:
# Test function
Jaccard_Similarity(scores_predict[:,0], scores_predict[:,4])

IndexError: index 4 is out of bounds for axis 1 with size 4

In [443]:
def Dice_Similarity(x,y):
    intersection = len(set(x) & set(y))
    return (2*intersection)/(len(x) + len(y))

In [444]:
# Test function
Dice_Similarity(scores_predict[:,0], scores_predict[:,4])

IndexError: index 4 is out of bounds for axis 1 with size 4

In [445]:
def Cosine_Similarity(x,y):
    dot_product = np.dot(x,y)
    x_norm = np.linalg.norm(x)
    y_norm = np.linalg.norm(y)
    return dot_product/(x_norm*y_norm)

In [446]:
# Test function
Cosine_Similarity(scores_predict[:,0], scores_predict[:,4])

IndexError: index 4 is out of bounds for axis 1 with size 4

In [447]:
def Adjusted_Cosine_Similarity(x,y):
    x_mean = np.mean(x)
    x_adjusted = x - x_mean
    
    y_mean = np.mean(y)
    y_adjusted = y - y_mean
    
    return Cosine_Similarity(x_adjusted, y_adjusted)

In [448]:
# Test function
Adjusted_Cosine_Similarity(scores_predict[:,0], scores_predict[:,4])

IndexError: index 4 is out of bounds for axis 1 with size 4

In [449]:
mae = 0
for idx, x in np.ndenumerate(scores_predict):
    if x == 0:
        predicted_score = predict_score_in_place(idx[0], idx[1], scores_predict)
        print("Predicted score in row: ", idx[0], " and in column: ", idx[1], " is: " + str(predicted_score))
        print("Actual score in row: ", idx[0], " and in column: ", idx[1], " is: " + str(scores[idx[0],idx[1]]))
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        actual_score = scores[idx[0],idx[1]]
        mae += np.absolute(predicted_score - actual_score)
print("The mean absolute error is: ", mae)
                
            

Predicted score in row:  2  and in column:  2  is: 5.1022559456622725
Actual score in row:  2  and in column:  2  is: 1.5071576530664377
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Predicted score in row:  2  and in column:  3  is: 5.023646145196072
Actual score in row:  2  and in column:  3  is: 3.61784713030248
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Predicted score in row:  3  and in column:  3  is: 4.79170852216785
Actual score in row:  3  and in column:  3  is: 3.868910392881798
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
The mean absolute error is:  5.923695436775479


In [437]:
def predict_score_in_place(n_row, n_col, score_matrix):
    k = 3
    similarities = []
    for j in range(len(score_matrix[n_row])):
        current_column = j
        if j == n_col:
            continue
        else:
            vector_01 = []
            vector_02 = []
            for i in range(len(score_matrix[:,current_column])):
                if score_matrix[i,current_column] != 0 and score_matrix[i,n_col] != 0:
                    vector_01.append(score_matrix[i,current_column])
                    vector_02.append(score_matrix[i,n_col])
            # Make some changes
            similarity = choose_comparision_method('Cosine', vector_01, vector_02)
            similarities.append((similarity, j))
    return calculate_score(score_matrix, similarities, n_row, k, 'Weighted Average')

In [434]:
def choose_comparision_method(name_of_method, vector_01, vector_02):
    if name_of_method == 'Jaccard':
        return Jaccard_Similarity(vector_01, vector_02)
    elif name_of_method == 'Dice':
        return Dice_Similarity(vector_01, vector_02)
    elif name_of_method == 'Cosine':
        return Cosine_Similarity(vector_01, vector_02)
    elif name_of_method == 'Adjusted Cosine':
        return Adjusted_Cosine_Similarity(vector_01, vector_02)

In [435]:
def calculate_score(score_matrix, similarity_matrix, row, k, method):
    similarity_matrix.sort(key=lambda x:x[0], reverse=True)
    sum_scores = 0
    if method == 'Simple Average':
        for i in range(k):
            column = similarity_matrix[i][1]
            score = score_matrix[row][column]
            sum_scores += score
        return sum_scores/k
    elif method == 'Weighted Average':
        sum_weights = 0;
        for i in range(k):
            column = similarity_matrix[i][1]
            weight = similarity_matrix[i][0]
            score = score_matrix[row][column]
            weighted_score = weight*score
            sum_scores += weighted_score
            sum_weights += weight
        return sum_scores/sum_weights